In [ ]:
source(paste0(dirname(getwd()),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))

### 0 - Read collected LILAC data

In [ ]:
lilac_qc <- fread( paste0(TMP_DIR, "lilac_qc.csv"))
lilac <- fread( paste0(TMP_DIR, "lilac.csv")) %>% filter(sampleId %in% (lilac_qc %>% filter(Status == "PASS") %>% pull(sampleId)))

### 1 - Prep biomarkers
- What to include, derive, filter?
- HLA heterozygosity?
- HLA LOH?

In [ ]:
lilac_base <-
lilac %>% 
 rw() %>% 
 mu(supertype = gsub("\\*", "", strsplit(Allele, ":")[[1]][1]), 
    receptor = strsplit(Allele, "\\*")[[1]][1], 
    loss = TumorCopyNumber < .5) %>% 
 ug() %>% 
 se(sampleId, receptor, Allele, supertype, TumorCopyNumber, loss)

In [ ]:
lilac_base

In [ ]:
hlas <- 
lilac |> 
  rowwise() |> 
  mutate( type = strsplit(Allele, ":")[[1]][1]) |>
  group_by( sampleId, type ) %>% 
  summarise( cn = sum(TumorCopyNumber)) |>
  spread(type, cn)

In [ ]:
hlas[is.na(hlas)] <- 0

In [ ]:
colnames(hlas) <- c("sampleId", paste0("lilac_", colnames(hlas)[-1]))

### 2 - Send it!

In [ ]:
fwrite(hlas, paste0(READY_DIR, "lilac_ready.csv"))